In [1]:
using AbstractAlgebra
AbstractAlgebra.PrettyPrinting.set_html_as_latex(true)

using AbstractAlgebra
AbstractAlgebra.PrettyPrinting.set_html_as_latex(true)

eqstr(x) = "\\ \\text{is " * string(x) * "}"
eqstr(x::SetElem) = "=" * sprint(x) do io, x; show(io, "text/latex", x) end

function dispeq(lhs, rhs)
    latexstr = "\$\\displaystyle $lhs " * eqstr(rhs) * "\$"
    display("text/html", latexstr)
end

function euclidean(f, g, stop = 1)
    while true
        r = mod(f, g)
        iszero(r) && return g
        degree(r) ≤ stop && return r
        f, g = g, r
    end
end

function rootdeg1(f)
    C = coefficients(f) |> collect
    -first(C)//last(C)
end

function commonroot(f, g)
    m = euclidean(f, g, 1)
    rootdeg1(m)
end

function revpoly(z, x, f)
    m = degree(f)
    C = coefficients(f)
    sum(a*z^(i-1)*x^(m-i+1) for (i, a) in enumerate(C))
end

function val(h::FracElem, θ, L)
    n = L(numerator(h)(θ))
    d = L(denominator(h)(θ))
    n//d
end

function testval(h::FracElem, θ, β, L)
    n = L(numerator(h)(θ))
    d = L(denominator(h)(θ))
    n == β*d
end

function calcallresults(z, x, f, g)
    f_plus = (-1)^degree(f)*f(z - x)
    F_plus = resultant(f_plus, g)
    β_plus = commonroot(g, f_plus)
    
    f_mult = (-1)^degree(f)*revpoly(z, x, f)
    F_mult = resultant(f_mult, g)
    β_mult = commonroot(g, f_mult)
    
    K = FractionField(base_ring(z))
    B1, α = K["α"]
    L1 = ResidueField(B1, numerator(f(z))(α))
    B2, β = L1["β"]
    L = ResidueField(B2, numerator(g(z))(β))
    
    val_F_plus = val(F_plus, α + β, L)
    test_β_plus = testval(β_plus, α + β, β, L)
    val_F_mult = val(F_mult, α * β, L)
    test_β_mult = testval(β_mult, α * β, β, L)
    
    S_plus = subresultant((-1)^degree(f)*f(z-x), g, 1)
    rootS_plus = rootdeg1(S_plus)
    
    F_plus, β_plus, F_mult, β_mult, val_F_plus, test_β_plus, val_F_mult, test_β_mult, S_plus, rootS_plus
end

function dispallresults(z, x, f, g)
    F_plus, β_plus, F_mult, β_mult, val_F_plus, test_β_plus, val_F_mult, test_β_mult, S_plus, rootS_plus =
        @time calcallresults(z, x, f, g)
    flush(stdout)
    dispeq("F_\\alpha(x)", f)
    dispeq("F_\\beta(x)", g)
    dispeq("R_{\\alpha + \\beta}(z)", F_plus)
    dispeq("R_{\\alpha + \\beta}(\\alpha + \\beta)", val_F_plus)
    dispeq("\\beta_\\mathrm{plus}(z)", β_plus)
    dispeq("\\beta_\\mathrm{plus}(\\alpha + \\beta) = \\beta", test_β_plus)
    dispeq("R_{\\alpha\\beta}(z)", F_mult)
    dispeq("R_{\\alpha\\beta}(\\alpha\\beta)", val_F_mult)
    dispeq("\\beta_\\mathrm{mult}(z)", β_mult)
    dispeq("\\beta_\\mathrm{mult}(\\alpha\\beta) = \\beta", test_β_mult)
    dispeq("\\text{1-subresultant}", S_plus)
    dispeq("\\text{root of 1-subresultant}", rootS_plus)
end

safecoeff(f, k) = 0 ≤ k ≤ degree(f) ? coeff(f, k) : zero(base_ring(f))

function subresultant_matrix(p::PolyElem{T}, q::PolyElem{T}, k) where T <: RingElement
    check_parent(p, q)
    R = parent(p)
    m = degree(p)
    n = degree(q)
    if length(p) == 0 || length(q) == k || m + n < 2k
       return zero_matrix(R, 0, 0)
    end
    M = zero_matrix(R, m + n - 2k, m + n - 2k)
    x = gen(R)
    for i in 1:n-k
        for j in 1:m+n-2k-1
           M[i, j] = safecoeff(p, m + (i-1) - (j-1))
        end
        M[i, end] = x^(n-k-i)*p
    end
    for i in 1:m-k
        for j in 1:m+n-2k-1
           M[n-k+i, j] = safecoeff(q, n + (i-1) - (j-1))
        end
        M[n-k+i, end] = x^(m-k-i)*q
    end
    return M
end

subresultant(p, q, k) = det(subresultant_matrix(p, q, k))

subresultant (generic function with 1 method)

In [2]:
R, (a, b, c, p, q, r, s) = ZZ["a", "b", "c", "p", "q", "r", "s"]
K = FractionField(R)
Rz, z = R["z"]
Kz = FractionField(Rz)
Rx, x = Kz["x"]

(Univariate Polynomial Ring in x over Fraction field of Univariate Polynomial Ring in z over Multivariate Polynomial Ring in 7 variables a, b, c, p, ..., s over Integers, x)

In [3]:
f = x^3 + a*x^2 + b*x + c
g = x^4 + p*x^3 + q*x^2 + r*x + s
subresultant_matrix(f, g, 0)

[1   a   b   c   0   0           x^6 + a*x^5 + b*x^4 + c*x^3]
[0   1   a   b   c   0           x^5 + a*x^4 + b*x^3 + c*x^2]
[0   0   1   a   b   c             x^4 + a*x^3 + b*x^2 + c*x]
[0   0   0   1   a   b                 x^3 + a*x^2 + b*x + c]
[1   p   q   r   s   0   x^6 + p*x^5 + q*x^4 + r*x^3 + s*x^2]
[0   1   p   q   r   s     x^5 + p*x^4 + q*x^3 + r*x^2 + s*x]
[0   0   1   p   q   r         x^4 + p*x^3 + q*x^2 + r*x + s]

In [4]:
S0 = sylvester_matrix(f, g)

[1   a   b   c   0   0   0]
[0   1   a   b   c   0   0]
[0   0   1   a   b   c   0]
[0   0   0   1   a   b   c]
[1   p   q   r   s   0   0]
[0   1   p   q   r   s   0]
[0   0   1   p   q   r   s]

In [5]:
subresultant(f, g, 0) == resultant(f, g) == det(S0)

true

In [6]:
subresultant_matrix(f, g, 1)

[1   a   b   c         x^5 + a*x^4 + b*x^3 + c*x^2]
[0   1   a   b           x^4 + a*x^3 + b*x^2 + c*x]
[0   0   1   a               x^3 + a*x^2 + b*x + c]
[1   p   q   r   x^5 + p*x^4 + q*x^3 + r*x^2 + s*x]
[0   1   p   q       x^4 + p*x^3 + q*x^2 + r*x + s]

In [7]:
S1 = Rx[
    1 a b c 0
    0 1 a b c*x
    0 0 1 a b*x+c
    1 p q r s*x
    0 1 p q r*x+s
]

[1   a   b   c         0]
[0   1   a   b       c*x]
[0   0   1   a   b*x + c]
[1   p   q   r       s*x]
[0   1   p   q   r*x + s]

In [8]:
subresultant(f, g, 1)

(-a^3*r + a^2*b*q + a^2*c*p + a^2*p*r - a^2*s - a*b^2*p - 2*a*b*c - a*b*p*q + 3*a*b*r - a*c*p^2 + a*p*s - a*q*r + b^3 + b^2*p^2 - 2*b^2*q + b*c*p - 2*b*p*r + b*q^2 + b*s + c^2 + c*p*q - 2*c*r - q*s + r^2)*x - a^3*s + a^2*c*q + a^2*p*s - a*b*c*p + 2*a*b*s - a*c^2 - a*c*p*q + a*c*r - a*q*s + b^2*c + b*c*p^2 - 2*b*c*q - b*p*s + c^2*p - c*p*r + c*q^2 - c*s + r*s

In [9]:
subresultant(f, g, 1) == det(S1)

true

In [10]:
S2 = Rx[
    1 a b*x^2+c*x
    0 1 a*x^2+b*x+c
    1 p q*x^2+r*x+s
]

[1   a       b*x^2 + c*x]
[0   1   a*x^2 + b*x + c]
[1   p   q*x^2 + r*x + s]

In [11]:
subresultant(f, g, 2)

(a^2 - a*p - b + q)*x^2 + (a*b - b*p - c + r)*x + a*c - c*p + s

In [12]:
subresultant(f, g, 2) == det(S2)

true

In [13]:
f = x^3 - a
g = x^4 - p
dispallresults(z, x, f, g)

  2.107416 seconds (1.87 M allocations: 115.317 MiB, 1.75% gc time, 97.91% compilation time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{4} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} - 4 a z^{9} - 3 p z^{8} + 6 a^{2} z^{6} - 48 a p z^{5} + 3 p^{2} z^{4} - 4 a^{3} z^{3} - 30 a^{2} p z^{2} - 12 a p^{2} z + a^{4} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a p}{z^{3}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(10 z^{6} + 16 a z^{3} + 6 p z^{2} + a^{2}\right) x - 6 z^{7} + 3 a z^{4} - 10 p z^{3} + 3 a^{2} z + a p$

$\displaystyle \text{root of 1-subresultant} =\frac{6 z^{7} - 3 a z^{4} + 10 p z^{3} - 3 a^{2} z - a p}{10 z^{6} + 16 a z^{3} + 6 p z^{2} + a^{2}}$

In [14]:
subresultant_matrix(-f(z-x), g, 1)

[1   -3*z   3*z^2   -z^3 + a   x^5 - 3*z*x^4 + 3*z^2*x^3 + (-z^3 + a)*x^2]
[0      1    -3*z      3*z^2     x^4 - 3*z*x^3 + 3*z^2*x^2 + (-z^3 + a)*x]
[0      0       1       -3*z            x^3 - 3*z*x^2 + 3*z^2*x - z^3 + a]
[1      0       0          0                                    x^5 - p*x]
[0      1       0          0                                      x^4 - p]

In [15]:
s1 = Rx[
    1 -3z 3z^2 -z^3+a 0
    0 1   -3z  3z^2   (-z^3+a)*x
    0 0   1    -3z    3z^2*x-z^3+a
    1 0   0    0      -p*x
    0 1   0    0      -p
]

[1   -3*z   3*z^2   -z^3 + a                   0]
[0      1    -3*z      3*z^2        (-z^3 + a)*x]
[0      0       1       -3*z   3*z^2*x - z^3 + a]
[1      0       0          0                -p*x]
[0      1       0          0                  -p]

In [16]:
S_plus = subresultant(-f(z-x), g, 1)

(10*z^6 + 16*a*z^3 + 6*p*z^2 + a^2)*x - 6*z^7 + 3*a*z^4 - 10*p*z^3 + 3*a^2*z + a*p

In [17]:
subresultant(-f(z-x), g, 1) == det(s1)

true

In [18]:
rootS_plus = rootdeg1(S_plus)
dispeq("\\text{root of 1-subresultant}", rootS_plus)

$\displaystyle \text{root of 1-subresultant} =\frac{6 z^{7} - 3 a z^{4} + 10 p z^{3} - 3 a^{2} z - a p}{10 z^{6} + 16 a z^{3} + 6 p z^{2} + a^{2}}$

In [19]:
f = x^3 - a
g = x^3 + p*x + q
dispallresults(z, x, f, g)

  0.069655 seconds (557.96 k allocations: 35.389 MiB, 21.67% gc time, 45.75% compilation time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{3} + p x + q$

$\displaystyle R_{\alpha + \beta}(z) =z^{9} + 3 p z^{7} + \left(-3 a + 3 q\right) z^{6} + 3 p^{2} z^{5} + \left(3 a p + 6 p q\right) z^{4} + \left(3 a^{2} + 21 a q + p^{3} + 3 q^{2}\right) z^{3} + \left(-3 a p^{2} + 3 p^{2} q\right) z^{2} + \left(-6 a^{2} p + 3 a p q + 3 p q^{2}\right) z - a^{3} + 3 a^{2} q - a p^{3} - 3 a q^{2} + q^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-z^{3} - a q}{a p}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(6 z^{4} + 3 p z^{2} + \left(3 a - 3 q\right) z + p^{2}\right) x - 3 z^{5} + p z^{3} + \left(3 a + 6 q\right) z^{2} - a p + p q$

$\displaystyle \text{root of 1-subresultant} =\frac{3 z^{5} - p z^{3} + \left(-3 a - 6 q\right) z^{2} + a p - p q}{6 z^{4} + 3 p z^{2} + \left(3 a - 3 q\right) z + p^{2}}$

In [20]:
f = x^3 - a
g = x^3 + p*x + q
dispallresults(z, x, g, f)

  0.040583 seconds (550.30 k allocations: 35.159 MiB, 36.41% gc time, 1.32% compilation time)


$\displaystyle F_\alpha(x) =x^{3} + p x + q$

$\displaystyle F_\beta(x) =x^{3} - a$

$\displaystyle R_{\alpha + \beta}(z) =z^{9} + 3 p z^{7} + \left(-3 a + 3 q\right) z^{6} + 3 p^{2} z^{5} + \left(3 a p + 6 p q\right) z^{4} + \left(3 a^{2} + 21 a q + p^{3} + 3 q^{2}\right) z^{3} + \left(-3 a p^{2} + 3 p^{2} q\right) z^{2} + \left(-6 a^{2} p + 3 a p q + 3 p q^{2}\right) z - a^{3} + 3 a^{2} q - a p^{3} - 3 a q^{2} + q^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-\frac{a p z}{z^{3} + a q}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(6 z^{4} + 3 p z^{2} + \left(3 a - 3 q\right) z + p^{2}\right) x - 3 z^{5} - 4 p z^{3} + \left(-6 a - 3 q\right) z^{2} - p^{2} z + a p - p q$

$\displaystyle \text{root of 1-subresultant} =\frac{3 z^{5} + 4 p z^{3} + \left(6 a + 3 q\right) z^{2} + p^{2} z - a p + p q}{6 z^{4} + 3 p z^{2} + \left(3 a - 3 q\right) z + p^{2}}$

In [21]:
h1 = g(z-x) + f

3*z*x^2 + (-3*z^2 - p)*x + z^3 + p*z - a + q

In [22]:
(3z)^2*f - 3z*x*h1

(9*z^3 + 3*p*z)*x^2 + (-3*z^4 - 3*p*z^2 + (3*a - 3*q)*z)*x - 9*a*z^2

In [23]:
h2 = (3z)^2*f - (3z*x + 3z^2 + p)*h1

(6*z^4 + 3*p*z^2 + (3*a - 3*q)*z + p^2)*x - 3*z^5 - 4*p*z^3 + (-6*a - 3*q)*z^2 - p^2*z + a*p - p*q

In [24]:
subresultant(f, g(z-x), 1)

(6*z^4 + 3*p*z^2 + (3*a - 3*q)*z + p^2)*x - 3*z^5 - 4*p*z^3 + (-6*a - 3*q)*z^2 - p^2*z + a*p - p*q

In [25]:
f = x^2 - a
g = x^2 - p
dispallresults(z, x, f, g)

  0.034942 seconds (170.22 k allocations: 10.306 MiB, 82.78% compilation time)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{2} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{4} + \left(-2 a - 2 p\right) z^{2} + a^{2} - 2 a p + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

$\displaystyle \text{1-subresultant} =2 z x - z^{2} + a - p$

$\displaystyle \text{root of 1-subresultant} =\frac{z^{2} - a + p}{2 z}$

In [26]:
f = x^2 - a
g = x^3 - p
dispallresults(z, x, f, g)

  0.029501 seconds (225.83 k allocations: 14.092 MiB, 54.18% gc time)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{3} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{6} - 3 a z^{4} - 2 p z^{3} + 3 a^{2} z^{2} - 6 a p z - a^{3} + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a p}{z^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(3 z^{2} + a\right) x - 2 z^{3} + 2 a z - p$

$\displaystyle \text{root of 1-subresultant} =\frac{2 z^{3} - 2 a z + p}{3 z^{2} + a}$

In [27]:
f = x^2 - a
g = x^4 - p
dispallresults(z, x, f, g)

  0.017217 seconds (374.81 k allocations: 23.972 MiB)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{4} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{8} - 4 a z^{6} + \left(6 a^{2} - 2 p\right) z^{4} + \left(-4 a^{3} - 12 a p\right) z^{2} + a^{4} - 2 a^{2} p + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

$\displaystyle \text{1-subresultant} =\left(4 z^{3} + 4 a z\right) x - 3 z^{4} + 2 a z^{2} + a^{2} - p$

$\displaystyle \text{root of 1-subresultant} =\frac{3 z^{4} - 2 a z^{2} - a^{2} + p}{4 z^{3} + 4 a z}$

In [28]:
f = x^2 - a
g = x^5 - p
dispallresults(z, x, f, g)

  0.047918 seconds (720.37 k allocations: 47.043 MiB, 29.99% gc time)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{5} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{10} - 5 a z^{8} + 10 a^{2} z^{6} - 2 p z^{5} - 10 a^{3} z^{4} - 20 a p z^{3} + 5 a^{4} z^{2} - 10 a^{2} p z - a^{5} + p^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a^{2} p}{z^{4}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(5 z^{4} + 10 a z^{2} + a^{2}\right) x - 4 z^{5} + 4 a^{2} z - p$

$\displaystyle \text{root of 1-subresultant} =\frac{4 z^{5} - 4 a^{2} z + p}{5 z^{4} + 10 a z^{2} + a^{2}}$

In [29]:
f = x^3 - a
g = x^3 - p
dispallresults(z, x, f, g)

  0.042086 seconds (431.47 k allocations: 27.515 MiB, 34.73% gc time, 18.60% compilation time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{3} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{9} + \left(-3 a - 3 p\right) z^{6} + \left(3 a^{2} - 21 a p + 3 p^{2}\right) z^{3} - a^{3} - 3 a^{2} p - 3 a p^{2} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =-1$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is false}$

$\displaystyle \text{1-subresultant} =\left(6 z^{4} + \left(3 a + 3 p\right) z\right) x - 3 z^{5} + \left(3 a - 6 p\right) z^{2}$

$\displaystyle \text{root of 1-subresultant} =\frac{z^{4} + \left(-a + 2 p\right) z}{2 z^{3} + a + p}$

In [30]:
f = x^3 - a
g = x^4 - p
dispallresults(z, x, f, g)

  0.053949 seconds (913.13 k allocations: 59.708 MiB, 28.60% gc time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{4} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} - 4 a z^{9} - 3 p z^{8} + 6 a^{2} z^{6} - 48 a p z^{5} + 3 p^{2} z^{4} - 4 a^{3} z^{3} - 30 a^{2} p z^{2} - 12 a p^{2} z + a^{4} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a p}{z^{3}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(10 z^{6} + 16 a z^{3} + 6 p z^{2} + a^{2}\right) x - 6 z^{7} + 3 a z^{4} - 10 p z^{3} + 3 a^{2} z + a p$

$\displaystyle \text{root of 1-subresultant} =\frac{6 z^{7} - 3 a z^{4} + 10 p z^{3} - 3 a^{2} z - a p}{10 z^{6} + 16 a z^{3} + 6 p z^{2} + a^{2}}$

In [31]:
f = x^3 - a
g = x^5 - p
dispallresults(z, x, f, g)

  0.105654 seconds (1.78 M allocations: 117.389 MiB, 30.06% gc time)


$\displaystyle F_\alpha(x) =x^{3} - a$

$\displaystyle F_\beta(x) =x^{5} - p$

$\displaystyle R_{\alpha + \beta}(z) =z^{15} - 5 a z^{12} - 3 p z^{10} + 10 a^{2} z^{9} - 90 a p z^{7} - 10 a^{3} z^{6} + 3 p^{2} z^{5} - 135 a^{2} p z^{4} + 5 a^{4} z^{3} - 30 a p^{2} z^{2} - 15 a^{3} p z - a^{5} - p^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{z^{6}}{a^{2} p}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(15 z^{8} + 51 a z^{5} + 10 p z^{3} + 15 a^{2} z^{2} - a p\right) x - 10 z^{9} - 6 a z^{6} - 15 p z^{4} + 15 a^{2} z^{3} + 6 a p z + a^{3}$

$\displaystyle \text{root of 1-subresultant} =\frac{10 z^{9} + 6 a z^{6} + 15 p z^{4} - 15 a^{2} z^{3} - 6 a p z - a^{3}}{15 z^{8} + 51 a z^{5} + 10 p z^{3} + 15 a^{2} z^{2} - a p}$

In [32]:
f = x^3 + a*x + b
g = x^3 + p*x + q
dispallresults(z, x, f, g)

  0.050997 seconds (688.92 k allocations: 43.092 MiB, 30.85% gc time, 1.55% compilation time)


$\displaystyle F_\alpha(x) =x^{3} + a x + b$

$\displaystyle F_\beta(x) =x^{3} + p x + q$

$\displaystyle R_{\alpha + \beta}(z) =z^{9} + \left(3 a + 3 p\right) z^{7} + \left(3 b + 3 q\right) z^{6} + \left(3 a^{2} + 3 a p + 3 p^{2}\right) z^{5} + \left(6 a b - 3 a q - 3 b p + 6 p q\right) z^{4} + \left(a^{3} + a^{2} p + a p^{2} + 3 b^{2} - 21 b q + p^{3} + 3 q^{2}\right) z^{3} + \left(3 a^{2} b + 3 a^{2} q - 6 a b p - 6 a p q + 3 b p^{2} + 3 p^{2} q\right) z^{2} + \left(a^{3} p - 2 a^{2} p^{2} + 3 a b^{2} - 3 a b q + a p^{3} - 6 a q^{2} - 6 b^{2} p - 3 b p q + 3 p q^{2}\right) z + a^{3} q + a^{2} b p - 2 a^{2} p q - 2 a b p^{2} + a p^{2} q + b^{3} + 3 b^{2} q + b p^{3} + 3 b q^{2} + q^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-b p z^{3} + a^{2} q z^{2} + b^{2} p q}{a z^{4} - a^{2} p z^{2} - a b q z - b^{2} p^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(6 z^{4} + \left(3 a + 3 p\right) z^{2} + \left(-3 b - 3 q\right) z + a^{2} - 2 a p + p^{2}\right) x - 3 z^{5} + \left(-4 a + p\right) z^{3} + \left(-3 b + 6 q\right) z^{2} + \left(-a^{2} + a p\right) z - a b - a q + b p + p q$

$\displaystyle \text{root of 1-subresultant} =\frac{3 z^{5} + \left(4 a - p\right) z^{3} + \left(3 b - 6 q\right) z^{2} + \left(a^{2} - a p\right) z + a b + a q - b p - p q}{6 z^{4} + \left(3 a + 3 p\right) z^{2} + \left(-3 b - 3 q\right) z + a^{2} - 2 a p + p^{2}}$

In [33]:
f = x^2 - a
g = x^3 + p*x^2 + q*x + r
dispallresults(z, x, f, g)

  0.012118 seconds (265.35 k allocations: 16.502 MiB)


$\displaystyle F_\alpha(x) =x^{2} - a$

$\displaystyle F_\beta(x) =x^{3} + p x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =z^{6} + 2 p z^{5} + \left(-3 a + p^{2} + 2 q\right) z^{4} + \left(-4 a p + 2 p q + 2 r\right) z^{3} + \left(3 a^{2} - 2 a p^{2} + 2 p r + q^{2}\right) z^{2} + \left(2 a^{2} p - 2 a p q + 6 a r + 2 q r\right) z - a^{3} + a^{2} p^{2} - 2 a^{2} q + 2 a p r - a q^{2} + r^{2}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{-p z^{2} - a r}{z^{2} + a q}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(3 z^{2} + 2 p z + a + q\right) x - 2 z^{3} - p z^{2} + 2 a z + a p + r$

$\displaystyle \text{root of 1-subresultant} =\frac{2 z^{3} + p z^{2} - 2 a z - a p - r}{3 z^{2} + 2 p z + a + q}$

In [34]:
f = x^3 + a*x + b
g = x^4 + p*x^2 + q*x + r
dispallresults(z, x, f, g)

  2.716055 seconds (6.46 M allocations: 290.173 MiB, 4.57% gc time)


$\displaystyle F_\alpha(x) =x^{3} + a x + b$

$\displaystyle F_\beta(x) =x^{4} + p x^{2} + q x + r$

$\displaystyle R_{\alpha + \beta}(z) =z^{12} + \left(4 a + 3 p\right) z^{10} + \left(4 b + 3 q\right) z^{9} + \left(6 a^{2} + 6 a p + 3 p^{2} + 3 r\right) z^{8} + \left(12 a b + 6 p q\right) z^{7} + \left(4 a^{3} + 4 a^{2} p + 4 a p^{2} - 8 a r + 6 b^{2} - 18 b q + p^{3} + 6 p r + 3 q^{2}\right) z^{6} + \left(12 a^{2} b - 6 a b p + 6 b p^{2} - 48 b r + 3 p^{2} q + 6 q r\right) z^{5} + \left(a^{4} + 2 a^{3} p - a^{2} p^{2} + 8 a^{2} r + 12 a b^{2} - 27 a b q + 2 a p^{3} - 12 a p r - 3 a q^{2} - 9 b^{2} p + 3 b p q + 3 p^{2} r + 3 p q^{2} + 3 r^{2}\right) z^{4} + \left(4 a^{3} b + 4 a^{3} q - 4 a^{2} b p - 8 a^{2} p q + 2 a b p^{2} - 4 a b r + 4 a p^{2} q - 16 a q r + 4 b^{3} - 18 b^{2} q + 2 b p^{3} - 24 b p r + 6 b q^{2} + 6 p q r + q^{3}\right) z^{3} + \left(a^{4} p - 2 a^{3} p^{2} + 4 a^{3} r + 6 a^{2} b^{2} + a^{2} p^{3} - 6 a^{2} q^{2} - 12 a b^{2} p - 9 a b p q + 3 a p q^{2} - 12 a r^{2} + 3 b^{2} p^{2} + 30 b^{2} r + 3 b p^{2} q + 6 b q r + 3 p r^{2} + 3 q^{2} r\right) z^{2} + \left(a^{4} q + 2 a^{3} b p - 2 a^{3} p q - 4 a^{2} b p^{2} - 4 a^{2} b r + a^{2} p^{2} q - 4 a^{2} q r + 4 a b^{3} + 2 a b p^{3} + 6 a b p r - 3 a b q^{2} + a q^{3} - 6 b^{3} p - 3 b^{2} p q - 6 b p^{2} r + 3 b p q^{2} + 12 b r^{2} + 3 q r^{2}\right) z + a^{4} r + a^{3} b q - 2 a^{3} p r + a^{2} b^{2} p - 2 a^{2} b p q + a^{2} p^{2} r + 2 a^{2} r^{2} - 2 a b^{2} p^{2} + 4 a b^{2} r + a b p^{2} q + 5 a b q r - 2 a p r^{2} + a q^{2} r + b^{4} + 3 b^{3} q + b^{2} p^{3} - 3 b^{2} p r + 3 b^{2} q^{2} - 3 b p q r + b q^{3} + r^{3}$

$\displaystyle R_{\alpha + \beta}(\alpha + \beta) =0$

...

$\displaystyle R_{\alpha\beta}(\alpha\beta) =0$

$\displaystyle \beta_\mathrm{mult}(z) =\frac{a z^{7} - a^{2} p z^{5} - a b q z^{4} + \left(a^{3} r - b^{2} p^{2} + b^{2} r\right) z^{3} + a b^{2} p r z - b^{3} q r}{b z^{6} + \left(-a^{3} q - 2 b^{2} q\right) z^{3} - a^{2} b r z^{2} - a b^{2} p q z - b^{3} p r + b^{3} q^{2}}$

$\displaystyle \beta_\mathrm{mult}(\alpha\beta) = \beta \ \text{is true}$

$\displaystyle \text{1-subresultant} =\left(10 z^{6} + \left(5 a + 9 p\right) z^{4} + \left(-16 b + 2 q\right) z^{3} + \left(4 a^{2} - 3 a p + 3 p^{2} - 6 r\right) z^{2} + \left(-4 a b - 7 a q + 3 p q\right) z + a^{3} - 2 a^{2} p + a p^{2} + a r + b^{2} + 2 b q - p r + q^{2}\right) x - 6 z^{7} + \left(-9 a - 3 p\right) z^{5} + \left(-3 b + 3 q\right) z^{4} + \left(-4 a^{2} - a p - p^{2} + 10 r\right) z^{3} + \left(3 a q - 3 b p\right) z^{2} + \left(-a^{3} + 2 a^{2} p - a p^{2} - 5 a r + 3 b^{2} + 3 b q + 3 p r\right) z - a^{2} b + 2 a b p - b p^{2} + b r + q r$

$\displaystyle \text{root of 1-subresultant} =\frac{6 z^{7} + \left(9 a + 3 p\right) z^{5} + \left(3 b - 3 q\right) z^{4} + \left(4 a^{2} + a p + p^{2} - 10 r\right) z^{3} + \left(-3 a q + 3 b p\right) z^{2} + \left(a^{3} - 2 a^{2} p + a p^{2} + 5 a r - 3 b^{2} - 3 b q - 3 p r\right) z + a^{2} b - 2 a b p + b p^{2} - b r - q r}{10 z^{6} + \left(5 a + 9 p\right) z^{4} + \left(-16 b + 2 q\right) z^{3} + \left(4 a^{2} - 3 a p + 3 p^{2} - 6 r\right) z^{2} + \left(-4 a b - 7 a q + 3 p q\right) z + a^{3} - 2 a^{2} p + a p^{2} + a r + b^{2} + 2 b q - p r + q^{2}}$